In [32]:
# FEniCS solver for 2D steady-state incompressible Navier-Stokes
# Lid-driven cavity flow with a non-uniform lid velocity profile

import dolfin as df
import matplotlib.pyplot as plt
import numpy as np

# -----------------------------------------------------------------------------
# 1. Problem Parameters
# -----------------------------------------------------------------------------
nu = 1.0 / 3000.0  # Kinematic viscosity (corresponding to Re=3000)
C0 = 10.0          # Constant for lid velocity profile
mesh_resolution = 64 # Mesh density (e.g., 64x64 elements)

# -----------------------------------------------------------------------------
# 2. Mesh and Function Space
# -----------------------------------------------------------------------------
# Create a unit square mesh for the domain Omega = [0, 1]^2
mesh = df.UnitSquareMesh(mesh_resolution, mesh_resolution)

# Define function spaces using Taylor-Hood elements (P2 for velocity, P1 for pressure)
# This element pair is stable for incompressible flow (satisfies the LBB condition)
V = df.VectorFunctionSpace(mesh, 'P', 2) # Velocity vector space
Q = df.FunctionSpace(mesh, 'P', 1)       # Pressure scalar space
W = df.MixedFunctionSpace([V, Q])        # Mixed function space for (u, p)

# -----------------------------------------------------------------------------
# 3. Boundary Conditions
# -----------------------------------------------------------------------------
# Define the geometric boundaries of the domain
walls = 'near(x[0], 0) || near(x[0], 1) || near(x[1], 0)'
lid   = 'near(x[1], 1)'

# Define the prescribed velocity profile on the lid using a FEniCS Expression
# u(x, 1) = 1 - cosh(C0*(x - 0.5)) / cosh(0.5*C0)
# v(x, 1) = 0
lid_velocity_str = ('1.0 - cosh(C0*(x[0] - 0.5)) / cosh(0.5*C0)', '0.0')
lid_velocity = df.Expression(lid_velocity_str, degree=2, C0=C0)

# Create Dirichlet boundary conditions
# No-slip (u=0, v=0) on the three stationary walls
bc_walls = df.DirichletBC(W.sub(0), df.Constant((0, 0)), walls)

# Prescribed velocity on the moving lid
bc_lid = df.DirichletBC(W.sub(0), lid_velocity, lid)

# Collect all boundary conditions into a list
bcs = [bc_walls, bc_lid]

# -----------------------------------------------------------------------------
# 4. Variational Problem Formulation
# -----------------------------------------------------------------------------
# Define trial and test functions for the mixed space W
(u, p) = df.TrialFunctions(W)
(v, q) = df.TestFunctions(W)

# Define a function to hold the solution (u, p)
w = df.Function(W)

# Split the solution function to handle the non-linear term
# u_sol will be the velocity field from the previous Newton iteration
(u_sol, p_sol) = df.split(w)

# Weak form of the steady-state, incompressible Navier-Stokes equations
# (u . grad(u)) . v * dx  --> Convective term
# nu * inner(grad(u), grad(v)) * dx --> Diffusive (viscous) term
# -p * div(v) * dx --> Pressure term
# -q * div(u) * dx --> Incompressibility constraint
F = (df.dot(df.dot(u_sol, df.grad(u_sol)), v) * df.dx +
     nu * df.inner(df.grad(u_sol), df.grad(v)) * df.dx -
     p_sol * df.div(v) * df.dx -
     q * df.div(u_sol) * df.dx)

# -----------------------------------------------------------------------------
# 5. Solve the Non-linear System
# -----------------------------------------------------------------------------
print("Solving the non-linear system...")

# The solve function automatically invokes a Newton solver for non-linear problems
# We can specify solver parameters for convergence control
df.solve(F == 0, w, bcs,
         solver_parameters={"newton_solver": {"absolute_tolerance": 1E-8,
                                              "relative_tolerance": 1E-7,
                                              "maximum_iterations": 20,
                                              "relaxation_parameter": 1.0}})

print("Solution complete.")

# Split the final mixed solution into individual functions for velocity and pressure
(u_final, p_final) = w.split()

# -----------------------------------------------------------------------------
# 6. Post-processing and Visualization
# -----------------------------------------------------------------------------
# Set plotting backend if needed (e.g., for different environments)
# plt.switch_backend('agg')

# --- Plot 1: Velocity Magnitude and Pressure ---
plt.figure(figsize=(12, 6))

# Plot velocity magnitude: |u| = sqrt(u_x^2 + u_y^2)
plt.subplot(1, 2, 1)
vel_mag_plot = df.plot(df.sqrt(df.dot(u_final, u_final)), title='Velocity Magnitude')
plt.colorbar(vel_mag_plot, fraction=0.046, pad=0.04)
plt.xlabel('$x$')
plt.ylabel('$y$')

# Plot pressure field
plt.subplot(1, 2, 2)
pressure_plot = df.plot(p_final, title='Pressure')
plt.colorbar(pressure_plot, fraction=0.046, pad=0.04)
plt.xlabel('$x$')

plt.tight_layout()
plt.savefig("navier_stokes_solution_mag_pressure.png")
print("Saved magnitude and pressure plot to 'navier_stokes_solution_mag_pressure.png'")

# --- Plot 2: Velocity Streamlines ---
plt.figure(figsize=(8, 8))
df.plot(u_final, title='Velocity Streamlines')
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.tight_layout()
plt.savefig("navier_stokes_solution_streamlines.png")
print("Saved streamlines plot to 'navier_stokes_solution_streamlines.png'")

# Show plots interactively
plt.show()

ModuleNotFoundError: No module named 'dolfin'